In [ ]:
pip install svglib reportlab pdf2image

In [1]:
import os
import zipfile

# Set the directory where the zip files are located
zip_files_directory = "C:\\Users\\jason\\Downloads\\wtfdata"  # Change this to your source directory
# Set the directory where all extracted files will go
destination_directory = 'C:\\Users\\jason\\Downloads\\faces'  # Change this to your destination folder

# Ensure the destination directory exists
os.makedirs(destination_directory, exist_ok=True)

# Iterate over all zip files in the directory
for zip_filename in os.listdir(zip_files_directory):
    if zip_filename.endswith('.zip'):
        zip_path = os.path.join(zip_files_directory, zip_filename)

        # Extract the reference number from the folder name (assuming "RefNo ABCDEF")
        ref_no = zip_filename.replace("RefNo ", "").replace(".zip", "").strip()

        # Open and extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            for file_info in zip_ref.infolist():
                file_extension = os.path.splitext(file_info.filename)[-1]  # Extract file extension
                new_filename = f"{ref_no}{file_extension}"  # Construct new file name
                extracted_path = os.path.join(destination_directory, new_filename)

                # Ensure no filename conflicts by adding an index if necessary
                counter = 1
                while os.path.exists(extracted_path):
                    new_filename = f"{ref_no}_{counter}{file_extension}"
                    extracted_path = os.path.join(destination_directory, new_filename)
                    counter += 1

                # Extract the file to the destination with the new name
                with zip_ref.open(file_info.filename) as source_file, open(extracted_path, 'wb') as target_file:
                    target_file.write(source_file.read())

print("Extraction and renaming complete.")

Extraction and renaming complete.


In [2]:
import csv
import json
import os
from collections import defaultdict

# Set file paths and column names manually
csv_file_path = "C:\\Users\\jason\\Downloads\\twes.csv"        # <-- Set your CSV file location
json_output_path = 'C:\\Users\\jason\\Downloads\\coa.json'      # <-- Set output JSON file location
name_column = '\ufeffnames'                               # <-- Set the CSV column name for "names"
image_column = 'images'                                   # <-- Set the CSV column name for "images"

data = {
    "img_folder_path": "faces",
    "details": []
}

# Use a dictionary to merge duplicate names and aggregate their images
merged_entries = defaultdict(list)

# Read CSV and build merged dictionary
with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        name = row[name_column]
        image_filename = row[image_column] + ".png"  # remove + ".png" if not needed
        merged_entries[name].append(image_filename)

# Build the final JSON structure
for name, images in merged_entries.items():
    entry = {
        "name": name,
        "images": images
    }
    data["details"].append(entry)

# Write to JSON
with open(json_output_path, 'w', encoding='utf-8') as jsonfile:
    json.dump(data, jsonfile, indent=4, ensure_ascii=False)

print(f"✅ Successfully created merged JSON from {csv_file_path} at: {json_output_path}")


✅ Successfully created merged JSON from C:\Users\jason\Downloads\twes.csv at: C:\Users\jason\Downloads\coa.json


In [3]:
import os
import shutil
from PIL import Image
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from pdf2image import convert_from_path

# Function to convert SVG to PNG using svglib and reportlab
def convert_svg_to_png(input_path, output_path):
    drawing = svg2rlg(input_path)
    renderPM.drawToFile(drawing, output_path, fmt="PNG")
    print(f"Converted {input_path} to {output_path}")

# Function to convert PDF to PNG
def convert_pdf_to_png(input_path, output_folder):
    images = convert_from_path(input_path)
    for i, image in enumerate(images):
        output_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(input_path))[0]}_{i+1}.png")
        image.save(output_path, 'PNG')
        print(f"Converted {input_path} page {i+1} to {output_path}")

# Function to convert other image formats (JPEG, JPG, RAW, WEBP) to PNG
def convert_image_to_png(input_path, output_path):
    try:
        with Image.open(input_path) as img:
            img.save(output_path, 'PNG')
        print(f"Converted {input_path} to {output_path}")
    except Exception as e:
        print(f"Error converting {input_path}: {e}")

# Function to copy existing PNG files
def copy_existing_png(input_path, output_path):
    try:
        shutil.copy2(input_path, output_path)
        print(f"Copied {input_path} to {output_path}")
    except Exception as e:
        print(f"Error copying {input_path}: {e}")

# Function to process files in a folder
def convert_images_in_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        if os.path.isdir(input_path):
            continue
        
        file_extension = os.path.splitext(filename)[1].lower()
        output_filename = os.path.splitext(filename)[0] + '.png'
        output_path = os.path.join(output_folder, output_filename)
        
        try:
            if file_extension in ['.jpg', '.jpeg', '.webp', '.raw']:
                convert_image_to_png(input_path, output_path)
            elif file_extension == '.svg':
                convert_svg_to_png(input_path, output_path)
            elif file_extension == '.pdf':
                convert_pdf_to_png(input_path, output_folder)
            elif file_extension == '.png':
                # If PNG file already exists, just copy it over
                copy_existing_png(input_path, os.path.join(output_folder, filename))
            else:
                print(f"Skipping unsupported file: {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Example usage
if __name__ == "__main__":
    input_folder = "C:\\Fusion\\Projects\\Real-time-FRS-2.0\\simpliFRy\\data\\faces"  # Replace with your input folder path
    output_folder = "C:\\Fusion\\Projects\\Real-time-FRS-2.0\\simpliFRy\\data\\bruh"  # Replace with your desired output folder path
    convert_images_in_folder(input_folder, output_folder)


Copied C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\faces\amanda.png to C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\bruh\amanda.png
Copied C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\faces\bryanchow.png to C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\bruh\bryanchow.png
Copied C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\faces\bryantan.png to C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\bruh\bryantan.png
Converted C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\faces\chow.jpg to C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\bruh\chow.png
Converted C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\faces\edward.jpg to C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\bruh\edward.png
Copied C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\faces\edward.png to C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\bruh\edward.png
Converted C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\faces\ethan.jpg to C:\Fusion\Projects\Real-t